## COMP 3400 MINI PROJECT: PREDICTING PRECIPITATION

#### Name: Kay-Salami Motolani Karima

#### Student ID: 20215668

In [4]:
import numpy as np
import pandas as pd

np.random.seed(42)  # For reproducibility

# Load the dataset
df = pd.read_csv("//Users//motolanikay-salami//Desktop//london_weather.csv")

# Remove the 'date' column as it's not needed for modeling
df = df.drop(columns=['date'])

# Drop rows with missing values to avoid errors in calculations
df = df.dropna()

print("Dataset loaded with shape:", df.shape)
print("Basic statistics of the data:\n", df.describe())

Dataset loaded with shape: (13843, 9)
Basic statistics of the data:
        cloud_cover      sunshine  global_radiation      max_temp  \
count  13843.00000  13843.000000      13843.000000  13843.000000   
mean       5.32818      4.262609        114.529148     14.951911   
std        2.03417      3.987488         87.758136      6.510220   
min        0.00000      0.000000         12.000000     -6.200000   
25%        4.00000      0.400000         39.000000     10.200000   
50%        6.00000      3.400000         89.000000     14.400000   
75%        7.00000      7.100000        180.000000     19.700000   
max        9.00000     15.700000        352.000000     37.900000   

          mean_temp      min_temp  precipitation       pressure    snow_depth  
count  13843.000000  13843.000000   13843.000000   13843.000000  13843.000000  
mean      11.085408      7.212302       1.667493  101538.493101      0.037853  
std        5.700936      5.319409       3.733947    1066.084413      0.545712 

In [6]:
# Features and target
X = df.drop(columns=['precipitation'])
y = df['precipitation']

In [8]:
split_idx = int(0.8 * len(df))

X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

print("Training set size:", X_train.shape[0], "samples")
print("Testing set size:", X_test.shape[0], "samples")

Training set size: 11074 samples
Testing set size: 2769 samples


In [10]:
# Add bias term (column of ones) to feature matrices
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train.values]
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test.values]

# Convert target vector to 2D array for calculations
y_train_np = y_train.values.reshape(-1, 1)

# Calculate theta (weights) using normal equation
theta = np.linalg.pinv(X_train_b.T @ X_train_b) @ X_train_b.T @ y_train_np

# Make predictions on test set
y_pred_baseline = (X_test_b @ theta).flatten()

# Calculate R² score manually
ss_res = np.sum((y_test.values - y_pred_baseline) ** 2)
ss_tot = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
baseline_r2 = 1 - ss_res / ss_tot

print("\nBaseline Linear Regression Model")
print("R² score on test set:", round(baseline_r2, 4))



Baseline Linear Regression Model
R² score on test set: 0.1306


In [12]:
# Initialize best score and current data for next steps
best_score = baseline_r2
current_X_train = X_train.copy()
current_X_test = X_test.copy()

# List to keep track of model performances
results_log = [("Baseline", baseline_r2)]

print("\nTrying data transformations to improve model:")

# Attempt 1: Standardization
mean_train = current_X_train.mean()
std_train = current_X_train.std()

X_train_std = (current_X_train - mean_train) / std_train
X_test_std = (current_X_test - mean_train) / std_train

Xb_train_std = np.c_[np.ones((X_train_std.shape[0], 1)), X_train_std.values]
Xb_test_std = np.c_[np.ones((X_test_std.shape[0], 1)), X_test_std.values]

theta_std = np.linalg.pinv(Xb_train_std.T @ Xb_train_std) @ Xb_train_std.T @ y_train.values.reshape(-1, 1)
y_pred_std = (Xb_test_std @ theta_std).flatten()

ss_res_std = np.sum((y_test.values - y_pred_std) ** 2)
ss_tot_std = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
r2_std = 1 - ss_res_std / ss_tot_std

print("Standardization R²:", round(r2_std, 4), "| Improvement:", r2_std > best_score)
results_log.append(("Standardization", r2_std))

if r2_std > best_score:
    best_score = r2_std
    current_X_train = X_train_std
    current_X_test = X_test_std
else:
    # Attempt 2: Min-Max Normalization
    min_train = current_X_train.min()
    max_train = current_X_train.max()

    X_train_norm = (current_X_train - min_train) / (max_train - min_train)
    X_test_norm = (current_X_test - min_train) / (max_train - min_train)

    Xb_train_norm = np.c_[np.ones((X_train_norm.shape[0], 1)), X_train_norm.values]
    Xb_test_norm = np.c_[np.ones((X_test_norm.shape[0], 1)), X_test_norm.values]

    theta_norm = np.linalg.pinv(Xb_train_norm.T @ Xb_train_norm) @ Xb_train_norm.T @ y_train.values.reshape(-1, 1)
    y_pred_norm = (Xb_test_norm @ theta_norm).flatten()

    ss_res_norm = np.sum((y_test.values - y_pred_norm) ** 2)
    ss_tot_norm = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
    r2_norm = 1 - ss_res_norm / ss_tot_norm

    print("Normalization R²:", round(r2_norm, 4), "| Improvement:", r2_norm > best_score)
    results_log.append(("Normalization", r2_norm))

    if r2_norm > best_score:
        best_score = r2_norm
        current_X_train = X_train_norm
        current_X_test = X_test_norm
    else:
        # Attempt 3: Max Abs Scaling
        max_abs_train = current_X_train.abs().max()

        X_train_maxabs = current_X_train / max_abs_train
        X_test_maxabs = current_X_test / max_abs_train

        Xb_train_maxabs = np.c_[np.ones((X_train_maxabs.shape[0], 1)), X_train_maxabs.values]
        Xb_test_maxabs = np.c_[np.ones((X_test_maxabs.shape[0], 1)), X_test_maxabs.values]

        theta_maxabs = np.linalg.pinv(Xb_train_maxabs.T @ Xb_train_maxabs) @ Xb_train_maxabs.T @ y_train.values.reshape(-1, 1)
        y_pred_maxabs = (Xb_test_maxabs @ theta_maxabs).flatten()

        ss_res_maxabs = np.sum((y_test.values - y_pred_maxabs) ** 2)
        ss_tot_maxabs = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
        r2_maxabs = 1 - ss_res_maxabs / ss_tot_maxabs

        print("Max Abs Scaling R²:", round(r2_maxabs, 4), "| Improvement:", r2_maxabs > best_score)
        results_log.append(("Max Abs Scaling", r2_maxabs))

        if r2_maxabs > best_score:
            best_score = r2_maxabs
            current_X_train = X_train_maxabs
            current_X_test = X_test_maxabs



Trying data transformations to improve model:
Standardization R²: 0.1306 | Improvement: True


In [12]:
# Initialize best score and current data for next steps
best_score = baseline_r2
current_X_train = X_train.copy()
current_X_test = X_test.copy()

# List to keep track of model performances
results_log = [("Baseline", baseline_r2)]

print("\nTrying data transformations to improve model:")

# Attempt 1: Standardization
mean_train = current_X_train.mean()
std_train = current_X_train.std()

X_train_std = (current_X_train - mean_train) / std_train
X_test_std = (current_X_test - mean_train) / std_train

Xb_train_std = np.c_[np.ones((X_train_std.shape[0], 1)), X_train_std.values]
Xb_test_std = np.c_[np.ones((X_test_std.shape[0], 1)), X_test_std.values]

theta_std = np.linalg.pinv(Xb_train_std.T @ Xb_train_std) @ Xb_train_std.T @ y_train.values.reshape(-1, 1)
y_pred_std = (Xb_test_std @ theta_std).flatten()

ss_res_std = np.sum((y_test.values - y_pred_std) ** 2)
ss_tot_std = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
r2_std = 1 - ss_res_std / ss_tot_std

print("Standardization R²:", round(r2_std, 4), "| Improvement:", r2_std > best_score)
results_log.append(("Standardization", r2_std))

if r2_std > best_score:
    best_score = r2_std
    current_X_train = X_train_std
    current_X_test = X_test_std
else:
    # Attempt 2: Min-Max Normalization
    min_train = current_X_train.min()
    max_train = current_X_train.max()

    X_train_norm = (current_X_train - min_train) / (max_train - min_train)
    X_test_norm = (current_X_test - min_train) / (max_train - min_train)

    Xb_train_norm = np.c_[np.ones((X_train_norm.shape[0], 1)), X_train_norm.values]
    Xb_test_norm = np.c_[np.ones((X_test_norm.shape[0], 1)), X_test_norm.values]

    theta_norm = np.linalg.pinv(Xb_train_norm.T @ Xb_train_norm) @ Xb_train_norm.T @ y_train.values.reshape(-1, 1)
    y_pred_norm = (Xb_test_norm @ theta_norm).flatten()

    ss_res_norm = np.sum((y_test.values - y_pred_norm) ** 2)
    ss_tot_norm = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
    r2_norm = 1 - ss_res_norm / ss_tot_norm

    print("Normalization R²:", round(r2_norm, 4), "| Improvement:", r2_norm > best_score)
    results_log.append(("Normalization", r2_norm))

    if r2_norm > best_score:
        best_score = r2_norm
        current_X_train = X_train_norm
        current_X_test = X_test_norm
    else:
        # Attempt 3: Max Abs Scaling
        max_abs_train = current_X_train.abs().max()

        X_train_maxabs = current_X_train / max_abs_train
        X_test_maxabs = current_X_test / max_abs_train

        Xb_train_maxabs = np.c_[np.ones((X_train_maxabs.shape[0], 1)), X_train_maxabs.values]
        Xb_test_maxabs = np.c_[np.ones((X_test_maxabs.shape[0], 1)), X_test_maxabs.values]

        theta_maxabs = np.linalg.pinv(Xb_train_maxabs.T @ Xb_train_maxabs) @ Xb_train_maxabs.T @ y_train.values.reshape(-1, 1)
        y_pred_maxabs = (Xb_test_maxabs @ theta_maxabs).flatten()

        ss_res_maxabs = np.sum((y_test.values - y_pred_maxabs) ** 2)
        ss_tot_maxabs = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
        r2_maxabs = 1 - ss_res_maxabs / ss_tot_maxabs

        print("Max Abs Scaling R²:", round(r2_maxabs, 4), "| Improvement:", r2_maxabs > best_score)
        results_log.append(("Max Abs Scaling", r2_maxabs))

        if r2_maxabs > best_score:
            best_score = r2_maxabs
            current_X_train = X_train_maxabs
            current_X_test = X_test_maxabs



Trying data transformations to improve model:
Standardization R²: 0.1306 | Improvement: True


In [14]:
# Feature Selection based on Correlation
print("\nStarting feature selection by correlation thresholds:")

for threshold in [0.95, 0.90, 0.85]:
    corr_matrix = current_X_train.corr().abs()
    upper_tri = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    high_corr = (corr_matrix.values * upper_tri) > threshold

    drop_indices = np.unique(np.where(high_corr)[1])
    drop_columns = corr_matrix.columns[drop_indices]

    X_train_fs = current_X_train.drop(columns=drop_columns)
    X_test_fs = current_X_test.drop(columns=drop_columns)

    Xb_train_fs = np.c_[np.ones((X_train_fs.shape[0], 1)), X_train_fs.values]
    Xb_test_fs = np.c_[np.ones((X_test_fs.shape[0], 1)), X_test_fs.values]

    theta_fs = np.linalg.pinv(Xb_train_fs.T @ Xb_train_fs) @ Xb_train_fs.T @ y_train.values.reshape(-1, 1)
    y_pred_fs = (Xb_test_fs @ theta_fs).flatten()

    ss_res_fs = np.sum((y_test.values - y_pred_fs) ** 2)
    ss_tot_fs = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
    r2_fs = 1 - ss_res_fs / ss_tot_fs

    print(f"Threshold {threshold} | Dropped {len(drop_columns)} features | R²: {round(r2_fs, 4)} | Improvement: {r2_fs > best_score}")
    results_log.append((f"Feature Selection thresh={threshold}", r2_fs))

    if r2_fs > best_score:
        best_score = r2_fs
        current_X_train = X_train_fs
        current_X_test = X_test_fs
        break



Starting feature selection by correlation thresholds:
Threshold 0.95 | Dropped 1 features | R²: 0.132 | Improvement: True


In [16]:
# Outlier Detection
print("\nTrying outlier removal to improve model:")

for z_threshold, min_features in [(3, 1), (3, 2), (2.5, 1)]:
    z_scores = (current_X_train - current_X_train.mean()) / current_X_train.std()
    outlier_flags = (np.abs(z_scores) > z_threshold).sum(axis=1) >= min_features

    X_train_clean = current_X_train[~outlier_flags]
    y_train_clean = y_train[~outlier_flags]

    Xb_train_clean = np.c_[np.ones((X_train_clean.shape[0], 1)), X_train_clean.values]
    Xb_test_current = np.c_[np.ones((current_X_test.shape[0], 1)), current_X_test.values]

    theta_clean = np.linalg.pinv(Xb_train_clean.T @ Xb_train_clean) @ Xb_train_clean.T @ y_train_clean.values.reshape(-1, 1)
    y_pred_clean = (Xb_test_current @ theta_clean).flatten()

    ss_res_clean = np.sum((y_test.values - y_pred_clean) ** 2)
    ss_tot_clean = np.sum((y_test.values - np.mean(y_test.values)) ** 2)
    r2_clean = 1 - ss_res_clean / ss_tot_clean

    print("Z-Score Threshold", z_threshold, "| Min Features", min_features, "| Removed", outlier_flags.sum(), "rows | R²:", round(r2_clean, 4), "| Improvement:", r2_clean > best_score)
    results_log.append(("Outlier removal z=" + str(z_threshold) + ", m=" + str(min_features), r2_clean))

    if r2_clean > best_score:
        best_score = r2_clean
        current_X_train = X_train_clean
        y_train = y_train_clean
        break


Trying outlier removal to improve model:
Z-Score Threshold 3 | Min Features 1 | Removed 147 rows | R²: 0.1324 | Improvement: True


In [18]:
print("\nSummary of model improvements:")
print("Baseline R²:", round(baseline_r2, 4))
print("Best R² after transformations and feature engineering:", round(best_score, 4))

print("\nDetailed steps and scores:")
for step_name, score_val in results_log:
    print("-", step_name + ":", "R² =", round(score_val, 4))



Summary of model improvements:
Baseline R²: 0.1306
Best R² after transformations and feature engineering: 0.1324

Detailed steps and scores:
- Baseline: R² = 0.1306
- Standardization: R² = 0.1306
- Feature Selection thresh=0.95: R² = 0.132
- Outlier removal z=3, m=1: R² = 0.1324
